<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語ベクトル
* いわゆるword2vec。
 * https://arxiv.org/abs/1301.3781
 * https://en.wikipedia.org/wiki/Word2vec
* 単語をベクトルとして表現したもの。
 * 単語埋め込み、単語分散表現、などとも言われる。
* 意味が近い単語はベクトルとしても近くなるように、作成されている。


* 今回はランタイムのタイプでGPUを選んでおいてください。

## 使いみち

### 単語の類似度評価
* ベクトルどうしの遠い近いを表す尺度はなんでも使える。
* 内積やコサイン類似度が使われることが多い。


### テキストのベクトル化
* 最もシンプルには、テキストに含まれるトークンの単語ベクトルの平均を取ればよい。
 * これをmean poolingと呼ぶ。
* word2vecの単語ベクトルを使ってテキストをベクトル化することは、最近はあまりない。
 * テキストのベクトル化に、今は深層学習言語モデルを使う。
 * 今回は最後に少しBERTを使った方法を紹介する。

## 単語ベクトルを作るアルゴリズム
* アルゴリズム自体の説明は、この授業では割愛する。
 * https://www.tensorflow.org/text/tutorials/word2vec

In [ ]:
!pip install datasets

## 準備

### WRIME: 主観と客観の感情分析データセット
* https://github.com/ids-cv/wrime

In [ ]:
from datasets import load_dataset

dataset = load_dataset("shunk031/wrime", "ver2")

* はじめから、train, validation, testの3つの集合に分けられている。

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = dataset[tag]["sentence"]
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

## spaCyの単語ベクトル
* 今回は英語テキストのみ。
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、中規模モデルをインストールする。

### 日本語中規模モデルのインストール
* https://spacy.io/models/ja#ja_core_news_md

In [ ]:
!python -m spacy download ja_core_news_md

### 全テキストのベクトル化
* spaCyではテキストを直接ベクトル化できる。
 * 内部では単語ベクトルの平均を求めている。

In [ ]:
from tqdm import tqdm
import numpy as np
import spacy

nlp = spacy.load('ja_core_news_md')

X = {}
for tag in tags:
  X[tag] = []
  for text in tqdm(texts[tag]):
    tokens = nlp(text)
    X[tag].append(tokens.vector)
  X[tag] = np.array(X[tag])

In [ ]:
X["validation"].shape

* ベクトル化した結果を保存しておく。

In [ ]:
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'wb') as f:
    np.save(f, X[tag])
  with open(f'wrime_{tag}_label.npy', 'wb') as f:
    np.save(f, labels[tag])

## 単語ベクトルを使った文書分類

* 保存しておいたテキストのベクトル表現を読み込む。

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
X["train"].shape

In [ ]:
labels["train"].shape

* ここではデータセットのラベルを2値に単純化する
 * ラベル0のテキストは取り除く。
 * negativeとpositiveは、それぞれ一つのクラスにまとめる。

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

# BERTによるテキストのベクトル化
* BERTの説明はしない。とりあえず使う。
* BERTを単にテキストをベクトル化するツールとして使う。

### 準備

In [ ]:
!pip install -q transformers fugashi[unidic-lite]

In [ ]:
!pip install sentence-transformers

### sentence BERTのロード
* 初回だけダウンロードに時間がかかる。
* 2回目以降は、ローカルに保存したモデルをロードするだけ。

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

* 全テキストのベクトル化

In [ ]:
X = {}
for tag in tags:
  X[tag] = embedder.encode(texts[tag])

* ベクトル化した結果を保存しておく。

In [ ]:
import numpy as np

for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'wb') as f:
    np.save(f, X[tag])

* ラベルを2値に単純化する。
 * 先ほどと同じ。

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

# 本日の課題
* 上で実行した感情分析の性能を上げてください。
* チューニングが済んだら、テストセットでscoreを計算してください。